In [1]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)
source('cleanup.R')

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

ERROR: Error in file(filename, "r", encoding = encoding): cannot open the connection


In [54]:
data <- read_csv("data/processed.hungarian.data", col_names = FALSE)
data <- rename(data,
       age = X1, 
       sex = X2, 
       chest_pain_type = X3,
       resting_blood_pressure = X4, 
       cholestorol = X5, 
       fasting_blood_sugar = X6,
       restecg = X7, 
       max_heart_rate = X8, 
       exercise_induced_angina = X9,
       oldpeak = X10, 
       slope = X11, 
       ca = X12,
       thal = X13, 
       num = X14)

data <- data |>
   select(-slope, -ca, -thal)

data <- data |>
   filter(resting_blood_pressure != '?', cholestorol != '?', fasting_blood_sugar != '?', restecg != '?', max_heart_rate != '?', exercise_induced_angina != '?', oldpeak != '?')

data$resting_blood_pressure <- as.numeric(data$resting_blood_pressure)
data$cholestorol <- as.numeric(data$cholestorol)
data$fasting_blood_sugar <- as.numeric(data$fasting_blood_sugar)
data$restecg <- as.numeric(data$restecg)
data$max_heart_rate <- as.numeric(data$max_heart_rate)
data$exercise_induced_angina <- as.numeric(data$exercise_induced_angina)
data$oldpeak <- as.numeric(data$oldpeak)

data_p <- data
data_p

data$sex <- as.factor(data$sex)
levels(data$sex)[levels(data$sex) == '0'] <- 'Female'
levels(data$sex)[levels(data$sex) == '1'] <- 'Male'

data$num <- as.factor(data$num)
levels(data$num)[levels(data$num) == '0'] <- 'No'
levels(data$num)[levels(data$num) == '1'] <- 'Yes'

data$fasting_blood_sugar <- as.factor(data$fasting_blood_sugar)
levels(data$fasting_blood_sugar)[levels(data$fasting_blood_sugar) == '0'] <- 'No'
levels(data$fasting_blood_sugar)[levels(data$fasting_blood_sugar) == '1'] <- 'Yes'

data$exercise_induced_angina <- as.factor(data$exercise_induced_angina)
levels(data$exercise_induced_angina)[levels(data$exercise_induced_angina) == '0'] <- 'No'
levels(data$exercise_induced_angina)[levels(data$exercise_induced_angina) == '1'] <- 'Yes'

data_r <- data
data_r

summary <- data_p |>
   group_by(num) |>
   summarize(across(age:oldpeak, mean))

summary <- summary |>
   select(-chest_pain_type, -restecg, -exercise_induced_angina)

summary

Rows: 294 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (9): X4, X5, X6, X7, X8, X9, X11, X12, X13
dbl (5): X1, X2, X3, X10, X14

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


age,sex,chest_pain_type,resting_blood_pressure,cholestorol,fasting_blood_sugar,restecg,max_heart_rate,exercise_induced_angina,oldpeak,num
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
28,1,2,130,132,0,2,185,0,0,0
29,1,2,120,243,0,0,160,0,0,0
30,0,1,170,237,0,1,170,0,0,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
56,1,4,155,342,1,0,150,1,3,1
58,0,2,180,393,0,0,110,1,1,1
65,1,4,130,275,0,1,115,1,1,1


age,sex,chest_pain_type,resting_blood_pressure,cholestorol,fasting_blood_sugar,restecg,max_heart_rate,exercise_induced_angina,oldpeak,num
<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<fct>,<dbl>,<fct>
28,Male,2,130,132,No,2,185,No,0,No
29,Male,2,120,243,No,0,160,No,0,No
30,Female,1,170,237,No,1,170,No,0,No
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
56,Male,4,155,342,Yes,0,150,Yes,3,Yes
58,Female,2,180,393,No,0,110,Yes,1,Yes
65,Male,4,130,275,No,1,115,Yes,1,Yes


num,age,sex,resting_blood_pressure,cholestorol,fasting_blood_sugar,max_heart_rate,oldpeak
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,46.79755,0.6503067,130.6074,237.7485,0.03680982,145.5706,0.2042945
1,49.38776,0.8775510,136.0204,267.2449,0.13265306,128.6735,1.2908163
